# 11주 은행
은행 컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.

* 입금액을 매개변수로 넣어서 입금하세요. amount가 실제 입금액과 일치하지 않으면 오류.

	function deposit(uint amount)

* 전액 인출 함수를 구현하세요. 인출 금액을 정하지 않아도 되므로 함수의 인자가 없다.

	function widthdrawAll()

* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)

	function getBalance() public view returns(uint, uint)

* 다른 계정으로 계좌이체 함수

	function forwardTo(address payable _receiver) public payable

* fallback()을 입금가능하도록 구현하세요. fallback이 호출되면 이벤트가 발생하여 전송자와 금액을 출력하세요.

이벤트 함수명은 PrintLog(address from, uint amount)



주피터 노트북에서 컴파일, 배포한 후, 아래 기능을 실행하여 그 결과를 출력하세요.

ganache 8335 띄우고, 노드로 하세요.

- 입금 11111 wei, 222 wei를 하고 컨트랙잔고 11333 wei 출력

- 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

- 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

- 컨트랙으로 111 송금하고, deposit()이 아니라 fallback으로 입금하세요. 컨트랙잔고 111 wei 출력.



## 계좌 조회

In [8]:
%%writefile src/web3accounts8335.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8335');
web3.eth.getAccounts(function(err,res) { console.log("(1) accounts: " + res); } );
my=[]
web3.eth.getAccounts(function(err, res) { for (x in res) { my.push(res[x]) } } );
console.log("(2) accounts from array: " + my[0]);

Writing src/web3accounts8335.js


In [3]:
!node src\web3accounts8335.js

(2) accounts from array: undefined
(1) accounts: 0x5854A437C9eE1ccED6fa05179D71699A5902F69a,0x702Fe077271b29bd562832ec8986Ab578E375Cc7,0x56762cAC1C6BfE48D656FaC0156dFD4b759FC9E0,0x28DB61a266f1eb2A4322CEb53c828E98210A1cad,0xd009DE3ed3726Eea9170d338F547486a83B4dc70,0xEc755dedCf8BACB8d4C8c0CD2b8588176169d820,0x43521B7030705071550297303d370008D7D899e6,0xD4F979cF5544CD694d82Bb20e1021b580cB84262,0xeF824996490D022E20849f54f6AA4956ccCC8DDD,0xe1200Cd397b6C0109B8987E2Bb4B0b91a2307Aba


## 함수 작성

In [1]:
%%writefile src/Bank.sol
pragma solidity ^0.6.4;

contract Bank {
    address payable owner;
    uint balance;
    event PrintLog(address from, uint amount);
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function widthdrawAll() public {
        owner.transfer(balance);
        balance = 0;
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function forwardTo(address payable _receiver) public payable {
        balance -= msg.value;
        owner.transfer(msg.value);
        _receiver.transfer(msg.value);
    }
    fallback () external payable {
        emit PrintLog(msg.sender, msg.value);
    }
}

Overwriting src/Bank.sol


In [4]:
!solc --gas --bin --abi src/Bank.sol


======= src/Bank.sol:Bank =======
Gas estimation:
construction:
   26082 + 154800 = 180882
external:
   fallback:	1507
   deposit(uint256):	21103
   forwardTo(address):	infinite
   getBalance():	1027
   widthdrawAll():	infinite
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506000600181905550610306806100686000396000f3fe6080604052600436106100435760003560e01c806312065fe0146100b157806327d8ad88146100e35780633c45937514610127578063b6b55f251461013e57610044565b5b7fac247564f70be8d5e06a6068acf596e7bbca33994b256697b38d49095d4f38053334604051808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018281526020019250505060405180910390a1005b3480156100bd57600080fd5b506100c661016c565b604051808381526020018281526020019250505060405180910390f35b610125600480360360208110156100f957600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff1

 --> src/Bank.sol:3:1:
  |
3 | contract Bank {
  | ^ (Relevant source part starts here and spans across multiple lines).
Note: The payable fallback function is defined here.
  --> src/Bank.sol:27:5:
   |
27 |     fallback () external payable {
   |     ^ (Relevant source part starts here and spans across multiple lines).



## 배포

In [3]:
!solc --optimize --combined-json abi,bin src/Bank.sol > src/Bank.json

 --> src/Bank.sol:3:1:
  |
3 | contract Bank {
  | ^ (Relevant source part starts here and spans across multiple lines).
Note: The payable fallback function is defined here.
  --> src/Bank.sol:27:5:
   |
27 |     fallback () external payable {
   |     ^ (Relevant source part starts here and spans across multiple lines).



In [5]:
%%writefile src/BankDeploy.js
var Web3=require('web3');
var web3;
var _abiBinJson = require('./Bank.json');
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abi);
_bin=_abiBinJson.contracts[contractName].bin;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8335"));
}
var _abiArray=[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"PrintLog","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"widthdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506000600181905550610306806100686000396000f3fe6080604052600436106100435760003560e01c806312065fe0146100b157806327d8ad88146100e35780633c45937514610127578063b6b55f251461013e57610044565b5b7fac247564f70be8d5e06a6068acf596e7bbca33994b256697b38d49095d4f38053334604051808373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018281526020019250505060405180910390a1005b3480156100bd57600080fd5b506100c661016c565b604051808381526020018281526020019250505060405180910390f35b610125600480360360208110156100f957600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061017b565b005b34801561013357600080fd5b5061013c61023d565b005b61016a6004803603602081101561015457600080fd5b81019080803590602001909291905050506102b1565b005b60008060015447915091509091565b346001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156101f2573d6000803e3d6000fd5b508073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f19350505050158015610239573d6000803e3d6000fd5b5050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc6001549081150290604051600060405180830381858888f193505050501580156102a6573d6000803e3d6000fd5b506000600181905550565b8034146102bd57600080fd5b806001600082825401925050819055505056fea26469706673582212202790db2f0f1c60453ae9e7f7e7d722264b57315068de195f9d0ad0bd2e4bef3a64736f6c63430006040033"
var _contract = new web3.eth.Contract(_abiArray);
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x5854A437C9eE1ccED6fa05179D71699A5902F69a", gas: 800000, gasPrice: '10000000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/BankDeploy.js


In [6]:
!node src/BankDeploy.js

0x2F5B319719441EC5e3435c3718796b51e308e5cb


## 사용

In [7]:
%%writefile src/BankUse.js
var Web3=require('web3');
var fs = require('fs');
var _abiBinJson = require('./Bank.json');      //importing a javascript file
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8335"));
contractName=Object.keys(_abiBinJson.contracts); //

_abi=_abiBinJson.contracts[contractName].abi; //use just as read from file
_abiArray=JSON.parse(JSON.stringify(_abi));
_abiArray=JSON.parse(_abi);      //JSON parsing needed!!


async function doIt() {
    var myBank = new web3.eth.Contract(_abiArray, '0x2F5B319719441EC5e3435c3718796b51e308e5cb');
    var event = myBank.events.PrintLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
        }
    });
    const accounts = await web3.eth.getAccounts();
    await myBank.methods.deposit(11111).send({from:accounts[0] , gas:259210, value: 11111});
    await myBank.methods.deposit(222).send({from:accounts[0] , gas:259210, value: 222});;
    
    // 11333 wei 출력
    const balanceJson = await myBank.methods.getBalance().call();
    balance = JSON.stringify(balanceJson);
    console.log("Balance: " + balance);
    //console.log("bankBalance: "+ bankBalance)
    
    // 333 wei 이체
    await myBank.methods.forwardTo(accounts[1]).send({from: accounts[0] , gas:259210, value: 333});
    
    //11000 wei 출력
    const balanceJson2 = await myBank.methods.getBalance().call();
    const balance2 = JSON.stringify(balanceJson2);
    console.log("Balance: " + balance2);
    
    // 333 wei 출력

    balance = await web3.eth.getBalance(accounts[1]);
    console.log("Balance of account[1]: " + balance);
   
    // widthdrawall 이전 잔고
    balancebefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before withdrawal: " + balancebefore);
    console.log("widthdrawing....");
    await myBank.methods.widthdrawAll().send({from:accounts[0]});
  
    // withdrawall 이후 잔고
    balanceafter = await web3.eth.getBalance(accounts[0]);
    subtract = balanceafter-balancebefore;
    console.log("Balance after withdrawal: " + balanceafter);
    console.log("BalanceAfter - BalanceBefore: "+ subtract);
    
    //fallback 함수 호출
    
    await web3.eth.sendTransaction({from:accounts[0], to:"0x2F5B319719441EC5e3435c3718796b51e308e5cb", gas: 230000, value: 111});
    const balanceJson3 = await myBank.methods.getBalance().call();
    const balance3 = JSON.stringify(balanceJson3);
    console.log("Balance: " + balance3);
    process.exit(1); //force exit to disconnect websocket
}

doIt()

Overwriting src/BankUse.js


In [8]:
!node src/BankUse.js

Balance: {"0":"11333","1":"11333"}
Balance: {"0":"11000","1":"11000"}
Balance of account[1]: 1000000000000000000333
Balance before withdrawal: 997540749515999988667
widthdrawing....
Balance after withdrawal: 997540688547999999667
BalanceAfter - BalanceBefore: -60967999897600
Event fired: {"0":"0x5854A437C9eE1ccED6fa05179D71699A5902F69a","1":"111","from":"0x5854A437C9eE1ccED6fa05179D71699A5902F69a","amount":"111"}
Balance: {"0":"0","1":"111"}
